# Train Notebook — Parking Slot Occupancy Classifier

This notebook converts your `train_model.py` training script into a **well-organized Jupyter notebook** with explanatory markdown cells and runnable code cells. Use this to step through data loading, model creation, training and evaluation **interactively**.

**Notes**
- The notebook mirrors the existing `train_model.py` logic but breaks it into small cells with explanations.
- Training can be run cell-by-cell. On a laptop it's recommended to use small epochs for testing first.
- Save the notebook and run in your `parking-project` conda environment (the same `parking-ai` env you used).


## 1) Imports and configuration
This cell imports required libraries and sets constants (data paths, model dir).

In [3]:
import os
import argparse
from glob import glob
from typing import List, Tuple

import time
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split

from PIL import Image
from torchvision import transforms, models

from tqdm.auto import tqdm

# Paths (adjust if needed)
DATA_ROOT = "/Users/thrinath/thrinath_files/INT395/parking-project/data/raw"
MODEL_DIR = "/Users/thrinath/thrinath_files/INT395/parking-project/models/trained"
os.makedirs(MODEL_DIR, exist_ok=True)

print("Notebook configured. DATA_ROOT =", DATA_ROOT, "MODEL_DIR =", MODEL_DIR)

Notebook configured. DATA_ROOT = /Users/thrinath/thrinath_files/INT395/parking-project/data/raw MODEL_DIR = /Users/thrinath/thrinath_files/INT395/parking-project/models/trained


## 2) Inspect dataset structure
Run the cell below to quickly inspect number of slots and counts per class. This helps confirm your `Emp` -> `empty` rename worked.

In [4]:
# Quick dataset inspection
from pathlib import Path
root = Path(DATA_ROOT)
slot_dirs = sorted([d for d in root.iterdir() if d.is_dir()])
print(f"Found {len(slot_dirs)} slot folders\n")
for sd in slot_dirs:
    empty_count = len(list((sd / 'empty').glob('*'))) if (sd / 'empty').exists() else 0
    occ_count = len(list((sd / 'occupied').glob('*'))) if (sd / 'occupied').exists() else 0
    print(f"{sd.name}: empty={empty_count}, occupied={occ_count}")

Found 6 slot folders

A1: empty=147, occupied=66
A2: empty=147, occupied=66
A3: empty=147, occupied=66
B1: empty=147, occupied=66
B2: empty=147, occupied=66
B3: empty=147, occupied=66


## 3) Dataset class
This cell defines the PyTorch `Dataset` used to load images and apply transforms. It mirrors the logic in `train_model.py`.

In [5]:
class ParkingSlotDataset(Dataset):
    def __init__(self,
                 root_dir: str,
                 classes: Tuple[str, ...] = ("empty", "occupied"),
                 img_size: int = 224,
                 train: bool = True):
        self.root_dir = root_dir
        self.classes = classes
        self.class_to_idx = {c: i for i, c in enumerate(classes)}
        self.img_size = img_size

        # Collect all image paths and labels
        self.samples: List[Tuple[str, int]] = []

        for slot_dir in sorted(os.listdir(root_dir)):
            slot_path = os.path.join(root_dir, slot_dir)
            if not os.path.isdir(slot_path):
                continue
            for cls in classes:
                cls_dir = os.path.join(slot_path, cls)
                if not os.path.isdir(cls_dir):
                    continue
                for ext in ("*.jpg", "*.jpeg", "*.png"):
                    for img_path in glob(os.path.join(cls_dir, ext)):
                        self.samples.append((img_path, self.class_to_idx[cls]))

        if not self.samples:
            raise RuntimeError(f"No images found in {root_dir} for classes {classes}")

        # Basic transforms: augmentation for train, light for val
        if train:
            self.transform = transforms.Compose([
                transforms.Resize((img_size, img_size)),
                transforms.RandomHorizontalFlip(),
                transforms.RandomRotation(5),
                transforms.ColorJitter(brightness=0.2, contrast=0.2),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225]),
            ])
        else:
            self.transform = transforms.Compose([
                transforms.Resize((img_size, img_size)),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225]),
            ])

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx: int):
        img_path, label = self.samples[idx]
        img = Image.open(img_path).convert("RGB")
        img = self.transform(img)
        return img, label

# Simple smoke test: create dataset with train=True and print sample count (do not load images yet)
print('ParkingSlotDataset defined.')

ParkingSlotDataset defined.


## 4) Build datasets and dataloaders
This cell creates train/validation split and dataloaders. It re-uses the dataset defined above. Adjust `img_size`, `batch_size`, and `val_split` as needed.

In [6]:
# Config - change as needed for quick runs
img_size = 224
batch_size = 16   # smaller batch on laptop if needed
val_split = 0.2
shuffle = True

# Build full dataset (train transforms by default)
full_dataset = ParkingSlotDataset(root_dir=DATA_ROOT, img_size=img_size, train=True)

# Compute sizes
val_size = int(len(full_dataset) * val_split)
train_size = len(full_dataset) - val_size
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

# Rebuild val_dataset with deterministic (no augment) transforms using selected indices
val_indices = train_dataset.indices if hasattr(train_dataset, 'indices') else None
# Note: random_split returns Subset with .indices attribute; but careful: train_dataset is a Subset.
val_indices = val_dataset.indices if hasattr(val_dataset, 'indices') else range(len(val_dataset))
val_samples = [full_dataset.samples[i] for i in val_indices]
val_dataset = ParkingSlotDataset(root_dir=DATA_ROOT, img_size=img_size, train=False)
val_dataset.samples = val_samples

print(f'Train samples: {len(train_dataset)}, Val samples: {len(val_dataset)}')

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=0)

# Show a batch shape
for xb, yb in train_loader:
    print('Batch X shape:', xb.shape, 'Batch y shape:', yb.shape)
    break

Train samples: 1018, Val samples: 254
Batch X shape: torch.Size([16, 3, 224, 224]) Batch y shape: torch.Size([16])


## 5) Create model and helper functions
This cell creates the MobileNetV3 small model, sets device, and provides helper functions for saving/loading.

In [7]:
def create_model(num_classes: int = 2):
    model = models.mobilenet_v3_small(weights=models.MobileNet_V3_Small_Weights.DEFAULT)
    in_features = model.classifier[3].in_features
    model.classifier[3] = nn.Linear(in_features, num_classes)
    return model

def get_device():
    if torch.backends.mps.is_available():
        return torch.device("mps")
    if torch.cuda.is_available():
        return torch.device("cuda")
    return torch.device("cpu")

device = get_device()
print('Using device:', device)

model = create_model(num_classes=2).to(device)
print('Model created. Number of parameters:', sum(p.numel() for p in model.parameters()))

def save_model(model, path):
    torch.save({
        'model_state_dict': model.state_dict(),
        'class_to_idx': {'empty': 0, 'occupied': 1},
        'img_size': img_size,
    }, path)
    print('Model saved to', path)

Using device: mps
Model created. Number of parameters: 1519906


## 6) Training loop
This cell contains a training loop. **Be careful** running it — it will train the model. For quick testing set `epochs=1` and small batch sizes. The code is the same as in your script.

In [8]:
def train_one_epoch(model, loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in tqdm(loader, desc="Train", leave=False):
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        _, preds = outputs.max(1)
        correct += preds.eq(labels).sum().item()
        total += labels.size(0)

    epoch_loss = running_loss / total
    epoch_acc = correct / total
    return epoch_loss, epoch_acc

def eval_one_epoch(model, loader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in tqdm(loader, desc="Val  ", leave=False):
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)

            running_loss += loss.item() * images.size(0)
            _, preds = outputs.max(1)
            correct += preds.eq(labels).sum().item()
            total += labels.size(0)

    epoch_loss = running_loss / total
    epoch_acc = correct / total
    return epoch_loss, epoch_acc

# Training runner (configure parameters here)
epochs = 3
lr = 1e-3
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)

best_val_acc = 0.0
best_model_path = os.path.join(MODEL_DIR, "slot_classifier_best.pth")

for epoch in range(1, epochs + 1):
    print(f"\\n[INFO] Epoch {epoch}/{epochs}")
    train_loss, train_acc = train_one_epoch(model, train_loader, criterion, optimizer, device)
    val_loss, val_acc = eval_one_epoch(model, val_loader, criterion, device)
    print(f"[INFO] Train loss: {train_loss:.4f}, acc: {train_acc:.4f}")
    print(f"[INFO] Val   loss: {val_loss:.4f}, acc: {val_acc:.4f}")

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        save_model(model, best_model_path)
        print(f"[INFO] New best model saved -> {best_model_path} (val_acc={val_acc:.4f})")

print('Training finished. Best val acc:', best_val_acc)

\n[INFO] Epoch 1/3


[INFO] Train loss: 0.0946, acc: 0.9686
[INFO] Val   loss: 0.0666, acc: 0.9921
Model saved to /Users/thrinath/thrinath_files/INT395/parking-project/models/trained/slot_classifier_best.pth
[INFO] New best model saved -> /Users/thrinath/thrinath_files/INT395/parking-project/models/trained/slot_classifier_best.pth (val_acc=0.9921)
\n[INFO] Epoch 2/3


[INFO] Train loss: 0.0435, acc: 0.9921
[INFO] Val   loss: 0.0518, acc: 0.9921
\n[INFO] Epoch 3/3


[INFO] Train loss: 0.0443, acc: 0.9902
[INFO] Val   loss: 0.3580, acc: 0.9803
Training finished. Best val acc: 0.9921259842519685


## 7) Save / Load model
If you trained the model above, the `save_model()` call saved the checkpoint. Use this cell to load the model weights for inference later.

In [9]:
# Load saved checkpoint (example)
ckpt_path = os.path.join(MODEL_DIR, "slot_classifier_best.pth")
if os.path.exists(ckpt_path):
    chk = torch.load(ckpt_path, map_location=device)
    model_loaded = create_model(num_classes=2).to(device)
    model_loaded.load_state_dict(chk['model_state_dict'])
    model_loaded.eval()
    print('Loaded model from', ckpt_path)
else:
    print('No checkpoint found at', ckpt_path)

Loaded model from /Users/thrinath/thrinath_files/INT395/parking-project/models/trained/slot_classifier_best.pth


## 8) Notes and next steps
- If training is slow on CPU, reduce `img_size` or `batch_size`.
- Consider `torch.compile()` (PyTorch 2+) for speedups if available.
- If you want, I can also convert this notebook into a cleaned PDF-ready report or wire it into your repo as `notebooks/training_experiments.ipynb`.